In [4]:

# ## 🚀 SFO 프로젝트 피처엔지니어링-time_series

# - 원본 CSV 파일 로드부터 `df_clean` 생성까지의 공통 전처리
# - 생성된 df_clean을 csv로 저장
#
# [시작 전 준비사항]
# 1. `사출성형.csv` 파일 경로 확인 (input_csv_file)
# 2. 필요한 라이브러리 설치: `pip install pandas numpy scikit-learn ipykernel `


import pandas as pd
import numpy as np
import os


In [ ]:

from sklearn.preprocessing import LabelEncoder
# --- 0. 설정 및 파일 경로 정의 ---
input_csv_file = '../../data/baseline_processed.csv'  
output_final_csv_file = '../../data/time_series.csv' # 전처리된 데이터 저장

target_column = 'T일 예정 수주량'


print("\n--- [1단계: Time series 피처 엔지니어링] ---")

df_time = None


try:
    if not os.path.exists(input_csv_file):
        raise FileNotFoundError(f"🚨 오류: 입력 파일 '{input_csv_file}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")

    df_raw = pd.read_csv(input_csv_file)
    print(f"1.1.공통 전처리 파일 로드 완료 (shape: {df_raw.shape})")


    df_raw['Date'] = pd.to_datetime(df_raw['Date'])

    df_manual_fe = df_raw.copy()
    
    print("1.2. 날짜 피처 생성 중 (Month, Day, DayOfWeek, Year)...")
    df_manual_fe['Year'] = df_manual_fe['Date'].dt.year
    df_manual_fe['Month'] = df_manual_fe['Date'].dt.month
    df_manual_fe['Day'] = df_manual_fe['Date'].dt.day
    df_manual_fe['DayOfWeek'] = df_manual_fe['Date'].dt.dayofweek

    # 주말 여부 (토=5, 일=6)
    df_manual_fe['is_weekend'] = df_manual_fe['DayOfWeek'].isin([5, 6]).astype(int)

    # 월초/월말 플래그 (출고 몰림/마감 몰림 가정)
    df_manual_fe['is_month_start3'] = (df_manual_fe['Day'] <= 3).astype(int)
    df_manual_fe['is_month_end3']   = (df_manual_fe['Day'] >= 28).astype(int)

    # 분기 정보
    df_manual_fe['Quarter'] = df_manual_fe['Date'].dt.quarter


    print("1.3. Product_Number 라벨 인코딩 중...")
    # Product_Number 라벨인코딩 
    le_product = LabelEncoder()
    df_manual_fe['Product_Number'] = le_product.fit_transform(df_manual_fe['Product_Number'])

    # 1.3 Lag / Rolling / 변동성 / 편차 피처 생성
    print("1.4. Lag / Rolling / 편차 / 변동성 피처 생성 중 (Product_Number별 groupby)...")

    
    print("1.5. Lag 및 Rolling 피처 생성 중 (Product_Number별 정렬)...")
    df_manual_fe = df_manual_fe.sort_values(by=['Product_Number', 'Date']).reset_index(drop=True)
    
    # groupby 준비
    g = df_manual_fe.groupby('Product_Number', group_keys=False)

    # T일 예정 수주량 기반 Lag[1,2,3,7,14]
    for lag in [1, 2, 3, 7, 14]:
        df_manual_fe[f'lag_{lag}_수주량'] = g[target_column].shift(lag)

    # T일 예상 수주량 기반 Lag[1,2,3,7,14]
    for lag in [1, 2, 3, 7, 14]:
        df_manual_fe[f'lag_{lag}_예상수주량'] =g['T일 예상 수주량'].shift(lag)

    # history_len: 이 SKU에서 몇 번째 기록인지 (0부터 시작)
    df_manual_fe['history_len'] = g.cumcount()

    # enough_history: 최소 14일 이상 누적된 지점부터는 lag_14 등도 안정적으로 존재
    # 이 마스크는 나중에 성능평가할 때 공정 비교 필터로 사용할 수 있음
    df_manual_fe['enough_history'] = (df_manual_fe['history_len'] >= 14).astype(int)

    # T일 예정 수주량 기반 Rolling(3, 7)
    df_manual_fe['rolling_mean_7_수주량'] = g[target_column].transform(lambda s : s.shift(1).rolling(7, min_periods=1).mean())
    df_manual_fe['rolling_mean_3_수주량'] = g[target_column].transform(lambda s : s.shift(1).rolling(3, min_periods=1).mean())
    
    # T일 예상 수주량 기반 Rolling(3, 7)
    df_manual_fe['rolling_mean_3_예상수주량'] = g['T일 예상 수주량'].transform(lambda s: s.shift(1).rolling(3, min_periods=1).mean())
    df_manual_fe['rolling_mean_7_예상수주량'] = g['T일 예상 수주량'].transform(lambda s: s.shift(1).rolling(7, min_periods=1).mean())


    # --- 편차(dev_from_roll): 현재값 - (최근 평균) ---
    # 지금 값이 평소 대비 얼마나 튀었는가? (급증/급락 감지)
    df_manual_fe['dev_from_roll3_수주량'] = df_manual_fe['lag_1_수주량'] - df_manual_fe['rolling_mean_3_수주량']
    df_manual_fe['dev_from_roll7_수주량'] = df_manual_fe['lag_1_수주량'] - df_manual_fe['rolling_mean_7_수주량']

    df_manual_fe['dev_from_roll3_예상수주량'] = df_manual_fe['T일 예상 수주량'] - df_manual_fe['rolling_mean_3_예상수주량']
    df_manual_fe['dev_from_roll7_예상수주량'] = df_manual_fe['T일 예상 수주량'] - df_manual_fe['rolling_mean_7_예상수주량']


    # --- 변동성(표준편차): 최근 7일 들쭉날쭉한 정도 ---
    # 예측 난이도 / 리스크 수준을 모델에 주는 역할
    df_manual_fe['roll7_std_수주량'] = g[target_column].transform(
        lambda s: s.shift(1).rolling(7, min_periods=1).std()
    )
    df_manual_fe['roll7_std_예상수주량'] = g['T일 예상 수주량'].transform(
        lambda s: s.shift(1).rolling(7, min_periods=1).std()
    )


    # --- 히스토리 길이 및 평가 마스크 ---
    # 1.6 마무리 정렬: 전체적으로 Date 기준으로 다시 정렬해서 export용/학습용 형태 맞춘다
    print("1.6. 최종 정렬 및 컬럼 정리 중...")
    df_time = df_manual_fe.copy()

    if not pd.api.types.is_datetime64_any_dtype(df_time['Date']):
        df_time['Date'] = pd.to_datetime(df_time['Date'], errors='coerce')

    df_time = df_time.sort_values(by='Date').reset_index(drop=True)

    df_time.to_csv(output_final_csv_file, index=False, encoding='utf-8-sig')
    print(f"Time series 피처 엔지니어링 완료! (df_time shape: {df_time.shape})")

except FileNotFoundError as e:
    print(f"🚨 {e}")
    df_time = None
except Exception as e:
    print(f"🚨 Time series 피처 엔지니어링 중 오류 발생: {e}")
    df_time = None




--- [1단계: Time series 피처 엔지니어링] ---
1.1.공통 전처리 파일 로드 완료 (shape: (10624, 19))
1.2. 날짜 피처 생성 중 (Month, Day, DayOfWeek, Year)...
1.3. Product_Number 라벨 인코딩 중...
1.4. Lag / Rolling / 편차 / 변동성 피처 생성 중 (Product_Number별 groupby)...
1.5. Lag 및 Rolling 피처 생성 중 (Product_Number별 정렬)...
1.6. 최종 정렬 및 컬럼 정리 중...
Time series 피처 엔지니어링 완료! (df_time shape: (10624, 49))


In [6]:
df_time.columns

Index(['Product_Number', 'T일 예정 수주량', '작년 T일 예정 수주량', 'T일 예상 수주량', 'T-1_예정',
       'T-1_예상', 'T-1_작년', 'T-2_예정', 'T-2_예상', 'T-2_작년', 'T-3_예정', 'T-3_예상',
       'T-3_작년', 'T-4_예정', 'T-4_예상', 'T-4_작년', 'Temperature', 'Humidity',
       'Date', 'Year', 'Month', 'Day', 'DayOfWeek', 'is_weekend',
       'is_month_start3', 'is_month_end3', 'Quarter', 'lag_1_수주량', 'lag_2_수주량',
       'lag_3_수주량', 'lag_7_수주량', 'lag_14_수주량', 'lag_1_예상수주량', 'lag_2_예상수주량',
       'lag_3_예상수주량', 'lag_7_예상수주량', 'lag_14_예상수주량', 'history_len',
       'enough_history', 'rolling_mean_7_수주량', 'rolling_mean_3_수주량',
       'rolling_mean_3_예상수주량', 'rolling_mean_7_예상수주량', 'dev_from_roll3_수주량',
       'dev_from_roll7_수주량', 'dev_from_roll3_예상수주량', 'dev_from_roll7_예상수주량',
       'roll7_std_수주량', 'roll7_std_예상수주량'],
      dtype='object')